# モデルの最大コンテキスト長を超えるテキストの埋め込み

OpenAIの埋め込みモデルは、最大長を超えるテキストを埋め込むことができません。最大長はモデルごとに異なり、トークン（token）単位で測定されます。トークン化に馴染みがない場合は、「トークンのカウント方法：tiktokenを使用する方法」をチェックしてみてください。

このノートブックでは、モデルの最大コンテキスト長を超えるテキストの扱い方を示します。埋め込みには`text-embedding-ada-002`の埋め込みを使用しますが、同じアイデアは他のモデルやタスクにも適用できます。埋め込みについて詳しく知りたい場合は、「OpenAI埋め込みガイド」を参照してください（https://beta.openai.com/docs/guides/embeddings）。


## 1. モデルのコンテキスト長

最初に、モデルを選択し、APIから埋め込みを取得するための関数を定義します。

In [2]:
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type

# 埋め込みモデルの設定
EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_CTX_LENGTH = 8191
EMBEDDING_ENCODING = 'cl100k_base'

# 無効なリクエストについてはリトライしないように設定します。これをデモンストレーションしたいからです。
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6), retry=retry_if_not_exception_type(openai.InvalidRequestError))
def get_embedding(text_or_tokens, model=EMBEDDING_MODEL):
    return openai.Embedding.create(input=text_or_tokens, model=model)["data"][0]["embedding"]


`text-embedding-ada-002` モデルは、`cl100k_base` エンコーディングを使用し、コンテキスト長が 8191 トークンです。この制限を超えるとエラーが発生することがわかります。

In [5]:
long_text = 'AGI ' * 5000
try:
    get_embedding(long_text)
except openai.InvalidRequestError as e:
    print(e)

This model's maximum context length is 8191 tokens, however you requested 10001 tokens (10001 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.


もちろん、特に大量の埋め込みをプログラムで処理する際に、これらのエラーを避けたいと思います。しかし、最大コンテキスト長よりも長いテキストに直面することがあるかもしれません。以下では、これらの長いテキストを処理するための主要なアプローチについて説明し、レシピを提供します：(1) テキストを最大許容長に単純に切り詰める方法、および(2) テキストをチャンクに分割し、各チャンクを個別に埋め込む方法。

## 1. 入力テキストの切り詰め

最もシンプルな解決策は、入力テキストを最大許容長に切り詰めることです。コンテキストの長さはトークンで測定されるため、まずテキストをトークン化する必要があります。APIはテキストまたはトークンの形式の入力を受け入れるため、適切なエンコーディングを使用していることに注意すれば、トークンを文字列形式に変換する必要はありません。以下はそのような切り詰め関数の例です。


In [6]:
import tiktoken

def truncate_text_tokens(text, encoding_name=EMBEDDING_ENCODING, max_tokens=EMBEDDING_CTX_LENGTH):
    """指定されたエンコーディングに従って、文字列を最大トークン数 `max_tokens` に切り詰めます。"""
    encoding = tiktoken.get_encoding(encoding_name)
    return encoding.encode(text)[:max_tokens]


以前の例は、今ではエラーなしで動作します。

In [7]:
truncated = truncate_text_tokens(long_text)
len(get_embedding(truncated))

1536

## 2. テキストのチャンキング

切り詰める方法は機能しますが、潜在的に関連するテキストを破棄することは明らかな欠点です。別のアプローチは、入力テキストをチャンクに分割し、それぞれのチャンクを個別に埋め込むことです。その後、チャンク埋め込みを個別に使用するか、各チャンクのサイズによって重み付けされた平均など、いくつかの方法でそれらを組み合わせることができます。

Pythonの公式クックブックから、シーケンスをチャンクに分割する関数を使います。[Python公式クックブック](https://docs.python.org/3/library/itertools.html#itertools-recipes)から関数を取得します。

In [8]:
from itertools import islice

def batched(iterable, n):
    """データを長さnのタプルにバッチ処理します。最後のバッチは短くなることがあります。"""
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('nは少なくとも1以上である必要があります')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch


文字列をトークンにエンコードし、それをチャンクに分割する関数を定義します。

In [9]:
def chunked_tokens(text, encoding_name, chunk_length):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    chunks_iterator = batched(tokens, chunk_length)
    yield from chunks_iterator

最終的に、入力テキストが最大コンテキスト長を超える場合でも安全に埋め込みリクエストを処理できる関数を書くことができます。この関数では、入力トークンをチャンクに分割し、各チャンクを個別に埋め込みます。`average` フラグを `True` に設定すると、チャンク埋め込みの加重平均が返されます。`False` に設定すると、修正されていないチャンク埋め込みのリストがそのまま返されます。

In [10]:
import numpy as np


def len_safe_get_embedding(text, model=EMBEDDING_MODEL, max_tokens=EMBEDDING_CTX_LENGTH, encoding_name=EMBEDDING_ENCODING, average=True):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(text, encoding_name=encoding_name, chunk_length=max_tokens):
        chunk_embeddings.append(get_embedding(chunk, model=model))
        chunk_lens.append(len(chunk))

    if average:
        chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
        chunk_embeddings = chunk_embeddings / np.linalg.norm(chunk_embeddings)  # normalizes length to 1
        chunk_embeddings = chunk_embeddings.tolist()
    return chunk_embeddings

再度、長い入力テキストを処理できるようになりました。

In [11]:
# average=Trueを設定すると、長いテキストに対して1つの平均埋め込みベクトルが得られます。
# このベクトルは、{len(average_embedding_vector)}次元のものです。
average_embedding_vector = len_safe_get_embedding(long_text, average=True)

# average=Falseを設定すると、各チャンクごとに1つの埋め込みベクトルが得られます。
# これにより、{len(chunks_embedding_vectors)}個の埋め込みベクトルが生成されます。
chunks_embedding_vectors = len_safe_get_embedding(long_text, average=False)

print(f"average=Trueを設定すると、長いテキストに対して1つの{len(average_embedding_vector)}次元の埋め込みベクトルが得られます。")
print(f"average=Falseを設定すると、{len(chunks_embedding_vectors)}個の埋め込みベクトルが生成され、各チャンクに対応します。")


average=Trueを設定すると、長いテキストに対して1つの1536次元の埋め込みベクトルが得られます。
average=Falseを設定すると、2個の埋め込みベクトルが生成され、各チャンクに対応します。


時折、テキストの意味を保持するのに役立つ場合があるため、段落の境界または文の境界でチャンクを分割することは意味があるかもしれません。